<a href="https://colab.research.google.com/github/ZYLion86/ImageDemo/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install scikit-image pillow opencv-python paddlepaddle paddlehub gradio

     |████████████████████████████████| 108.4 MB 21 kB/s 
     |████████████████████████████████| 212 kB 38.7 MB/s 
     |████████████████████████████████| 2.9 MB 41.5 MB/s 
     |████████████████████████████████| 1.2 MB 47.5 MB/s 
     |████████████████████████████████| 2.7 MB 44.0 MB/s 
     |████████████████████████████████| 2.6 MB 46.8 MB/s 
     |████████████████████████████████| 79 kB 8.2 MB/s 
     |████████████████████████████████| 12.2 MB 39.8 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 325 kB 55.6 MB/s 
     |████████████████████████████████| 64 kB 2.8 MB/s 
     |████████████████████████████████| 2.1 MB 49.4 MB/s 
     |████████████████████████████████| 197 kB 58.3 MB/s 
     |████████████████████████████████| 127 kB 49.3 MB/s 
     |████████████████████████████████| 69 kB 7.3 MB/s 
     |████████████████████████████████| 42 kB 918 kB/s 
     |████████████████████████████████| 211 kB 50.9 MB/s 
     |█████████████████